In [1]:
import pandas as pd

df = pd.read_csv("scraped_data_in_treatment/Alternative Medicine.csv")

In [2]:
alternative_medicine = df.groupby('thread_title')['comment_text'].apply(list).to_dict()
next(iter(alternative_medicine)),next(iter(alternative_medicine.values()))

('Anyone Else Out There Choosing 100 Alternative',
 ["He is, was a compatriot of Ernest Zundel. \xa0An an extremist holocast denier who was deported by Canada to Germany to face trial for his hate spreading. \xa0Zundel is an evil man. \xa0Tend to hang with similar types. \xa0I don't think BCO members want to associate themselves with such folk. \xa0Just sayin'. \xa0His theories have just as much validity as holocast denial, and we should, as a group disassociate ourselves from them.",
  'This discussion has been going on since at least February 2010 and maybe even 2007 ( I am not as good at searching this website as I should be). \xa0I thought I remembered this dust up.\xa0https://community.breastcancer.org/forum/7/topic/692216?page=2',
  'In response to the original question, of has anyone tried strictly Alternative solution to breast cancer: \xa0My grandmother was diagnosed with breast cancer in the early 70\'s. \xa0She was stage IV when diagnosed and the doctors had no suggestions f

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def extract_keywords_tfidf(thread_dict, top_n=5):
    """
    Extracts TF-IDF keywords for each thread in a dictionary (English).
    
    Parameters:
        thread_dict (dict): dictionary {thread_name: [list of sentences]}
        top_n (int): number of keywords to retrieve per thread
    
    Returns:
        dict: {thread_name: [list of keywords]}
    """
    # Merge sentences into a single text per thread
    documents = {thread: " ".join(comments) for thread, comments in thread_dict.items()}
    
    # TF-IDF with built-in English stopwords
    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(documents.values())
    
    # Convert to DataFrame for easier manipulation of scores
    df = pd.DataFrame(
        X.toarray(),
        index=[str(thread) for thread in documents.keys()],
        columns=vectorizer.get_feature_names_out()
    )
    
    # Extract top keywords for each thread
    keywords = {}
    for thread in df.index:
        sorted_words = df.loc[thread].sort_values(ascending=False)
        keywords[thread] = list(sorted_words.head(top_n).index)
    
    return keywords


In [4]:
keywords_alternative_medicine=extract_keywords_tfidf(alternative_medicine,top_n=8)

In [5]:
import ipywidgets as widgets
from IPython.display import display, Markdown

def visualize_thread_keywords(thread_keywords):
    """
    Create an interactive widget to display keywords per thread.
    
    Parameters:
        thread_keywords (dict): {thread_name: [list of keywords]}
    """
    # Dropdown with thread names
    dropdown = widgets.Dropdown(
        options=list(thread_keywords.keys()),
        description="Threads:",
        disabled=False,
        layout=widgets.Layout(width='400px')
    )

    # Output area to display keywords
    output = widgets.Output()

    # Function to update output when dropdown changes
    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            output.clear_output()
            thread_name = change['new']
            keywords = thread_keywords[thread_name]
            with output:
                display(Markdown(f"**Keywords for '{thread_name}':**"))
                display(Markdown(", ".join(keywords)))

    dropdown.observe(on_change)

    # Display dropdown and output
    display(dropdown, output)
    
    # Initialize first display
    dropdown.value = list(thread_keywords.keys())[0]


visualize_thread_keywords(keywords_alternative_medicine) 


Dropdown(description='Threads:', layout=Layout(width='400px'), options=('Anyone Else Out There Choosing 100 Al…

Output()

In [6]:
import os
import pandas as pd

# Dossier contenant les CSV
folder = "scraped_data_in_treatment"

# Dictionnaire pour stocker chaque DataFrame
csv_databases = {}

# Parcours de tous les fichiers CSV du dossier
for filename in os.listdir(folder):
    if filename.endswith(".csv"):
        path = os.path.join(folder, filename)
        # Nom de la database = nom du fichier sans extension
        db_name = os.path.splitext(filename)[0]
        csv_databases[db_name] = pd.read_csv(path)

# Exemple d'accès : csv_databases["Alternative Medicine"]
csv_databases.keys()

dict_keys(['Alternative Medicine', 'Breast Reconstruction', 'Chemotherapy - Before, During, and After', 'Clinical Trials, Research News, Podcasts, and Study Results', 'Complementary and Holistic Medicine and Treatment', 'Hormonal Therapy - Before, During, and After', 'Immunotherapy - Before, During, and After', 'Living Without Reconstruction After a Mastectomy', 'Managing Side Effects of Breast Cancer and Its Treatment', 'Radiation Therapy - Before, During, and After', 'Surgery - Before, During, and After', 'Targeted Therapy - Before, During, and After'])

In [7]:
csv_databases['Breast Reconstruction']

,thread_title,comment_text
0,Breast Reconstruction Revision,Hi! I am new to the forum. I was diagnosed in ...
1,Breast Reconstruction Revision,"Hello @marea123, and welcome to Breastcancer.o..."
2,Breast Reconstruction Revision,you’ll definitely want a plastic surgeon with ...
3,Breast Reconstruction Revision,"Hello, I'm adding to Marea's question in a way..."
4,Flap Reconstruction Decisions,Hello all! I was diagnosed in 2012 and had a ...
...,...,...
2343,Implant After Radiation,"Hello all,I had a mastectomy and then 16 round..."
2344,Implant After Radiation,"I had a bilateral, direct to implant mastectom..."
2345,Implant After Radiation,"I had a mastectomy in July, 2021 followed by 5..."
2346,Implant After Radiation,"Hi Brooklyn,I am probably too late in replying..."


In [8]:
df = csv_databases['Breast Reconstruction']
df
thread_dict = df.groupby('thread_title')['comment_text'].apply(list).to_dict()
thread_dict.keys()

dict_keys(['2 Yrs After Diep', '2018 Diep Surgery', '3 Step Reconstruction', '3D Tattoo Vs Nipple Reconstruction Or Both', 'Abdominal Bulge After Tram Flap', 'Acellular Dermal Matrix Adm With Your Mastecomy Reconstruction', 'Advice On Explant Diep Surgery', 'After Flap Reconstruction Complications More Likely In Women With Higher Bmi', 'Am I Crazy To Consider A Diep This Far Out', 'Any Regrets Nipple Sparing Or Skin Sparing Reconstruction', 'Anyone Else W Infection After Fat Transfer', 'Anyone Had A Strange Acting Infection After Tissue Expanders', 'Anyone Had Omental Flap Reconstruction', 'Anyone Have Experience With Cupping Massage', 'Anyone Heard Of The Swim Breast Reconstruction Technique', 'Anyone Lumpy After Diep', 'Anyone Need A Touch Up After Tattoo', 'Anyone Wiht Nipple Discharge After Nsm Told Possible Dcis Cells', 'Anyones Expander Rotate Swelling After Exchange', 'Athletics Exercise And Rehab After Masctectomy', 'Best Options For Pain Management', 'Bia Alcl Symptoms', 'Blac

In [9]:
# Appliquer extract_keywords_tfidf à chaque database
keywords_per_database = {}
for db_name, df in csv_databases.items():
    # Créer le dictionnaire {thread_title: [list of comment_text]}
    print(db_name)
    thread_dict = df.groupby('thread_title')['comment_text'].apply(list).to_dict()
    print('ok')
    # Extraire les mots-clés
    keywords_per_database[db_name] = extract_keywords_tfidf(thread_dict, top_n=8)

# keywords_per_database contient les mots-clés pour chaque database
# Exemple d'accès : keywords_per_database["Alternative Medicine"]

Alternative Medicine
ok
Breast Reconstruction
ok


TypeError: sequence item 0: expected str instance, float found